In [ ]:
import os
import yaml
import pypsa

import tz_pypsa as tza
from tz_pypsa.model import Model

## Build a base network from TZA-PyPSA-ASEAN

In [ ]:
network = (
    Model.load_model(
        'ASEAN', 
        frequency = '1h',
        select_nodes = ['MYSPE'], 
        years = [2030],
        backstop = False,
        set_global_constraints = False,
    )
)

## Define function to build a CFE network

In [ ]:
def PrepareNetworkForCFE(network, buses_with_ci_load):
    '''
    
    In this function, we take a PyPSA network and create a virtual subsystem representing 
        a C&I asset. Here, we loop through a set of buses in our brownfield network where
        we want to add a C&I load. We create a subsystem attached to each define bus, as 
        we do in the demo script / schematic.

    '''

    clean_carriers = ['photovoltaic-unspecified']
    clean_storages = ['lithium-ion']

    n = network.copy()

    # loop and add buses
    for bus in buses_with_ci_load:

        # define C&I bus
        ci_bus = f'{bus} C&I Grid'
        ci_storage = f'{bus} C&I Storage'

        # add bus for C&I system
        n.add(
            "Bus",
            ci_bus,
            x = n.buses.x.iloc[0] + 1,
            y = n.buses.y.iloc[0] + 1,       
        )

        # add bus for C&I storage
        n.add(
            "Bus",
            ci_storage,
            x = n.buses.x.iloc[0] - 1,
            y = n.buses.y.iloc[0] - 1,
        )

        # add C&I load
        n.add(
            "Load",
            f'{bus} C&I Load',
            bus = ci_bus,
            p_set = n.loads_t.p_set[bus] * 0.1,
        )

        # add clean generator for PPA
        n.add(
            "Generator",
            f'{bus} PPA Generation',
            bus = ci_bus,
            carrier = 'C&I Clean',
            p_nom = 0,
            p_nom_extendable = True,
            capital_cost = 1e12,
            marginal_cost = 1e3,
            p_max_pu = n.generators_t.p_max_pu.filter(regex='photo').values.flatten(),
        )

        # add clean storage for PPA
        n.add(
            "StorageUnit",
            f"{bus} PPA Storage",
            bus=ci_storage,
            p_nom=0,
            p_nom_extendable=True,
            cyclic_state_of_charge=True,
            capital_cost=1e9,
            marginal_cost=0,
            max_hours=4,
            efficiency_store=0.9,
            efficiency_dispatch=0.9,
        )

        # add links to connect Bus <-> C&I System
        n.add(
            "Link",
            f"{bus} Grid Imports",
            bus0=bus, 
            bus1=ci_bus, 
            p_nom=1e12,
            marginal_cost=1,
        )

        n.add(
            "Link",
            f"{bus} Grid Exports",
            bus0=ci_bus, 
            bus1=bus, 
            p_nom=1e12,
            marginal_cost=1,
        )

        # add links to connect C&I System <-> Storage
        n.add(
            "Link",
            f"{bus} StorageCharge",
            bus0=ci_bus, 
            bus1=ci_storage, 
            p_nom=0,
            p_nom_extendable=True,
            marginal_cost=1,
        )

        n.add(
            "Link",
            f"{bus} StorageDischarge",
            bus0=ci_storage, 
            bus1=ci_bus, 
            p_nom=0,
            p_nom_extendable=True,
            marginal_cost=1,
        )

    return n

## Run a brownfield network simulation

In [ ]:
brownfield = network.copy()

# prep for CFE (though nothing happens on C&I system in brownfield)
brownfield = PrepareNetworkForCFE(network, buses_with_ci_load=['MYSPE'])

# optimise
brownfield.optimize(solver_name='gurobi')

## Run a cfe simulation

In [ ]:
n

## Post-processing

In [ ]:
brownfield.statistics.energy_balance() / 1e5

In [ ]:
cfe.statistics.energy_balance()  / 1e5

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_dispatch(n, ci_label='C&I', timesteps=24):

    gen = n.generators_t.p.filter(regex=ci_label).sum(axis=1).iloc[0:timesteps].to_frame('PPA')

    if not n.storage_units.empty:
        sto = n.storage_units_t.p.filter(regex=ci_label).sum(axis=1).iloc[0:timesteps].to_frame('Storage')
        
    imports = n.links_t.p0.filter(regex='Import').sum(axis=1).iloc[0:timesteps].to_frame('Import')
    exports = n.links_t.p0.filter(regex='Export').sum(axis=1).iloc[0:timesteps].to_frame('Export').mul(-1)

    p_by_carrier = pd.concat([gen, sto, imports, exports], axis=1)

    fig, ax = plt.subplots(figsize=(6, 3))

    color_dict = {
        'PPA' : 'teal',
        'Storage' : 'peachpuff',
        'Import' : 'coral',
        'Export' : 'peru',
    }

    color = (
        p_by_carrier
        .columns
        .map(color_dict)
    )

    p_by_carrier.where(p_by_carrier > 0).iloc[0:timesteps].plot.area(
        ax=ax,
        linewidth=0,
        color=color,
    )

    charge = p_by_carrier.where(p_by_carrier < 0).dropna(how="all", axis=1)

    if not charge.empty:
        charge.plot.area(
            ax=ax,
            linewidth=0,
            color=charge.columns.map(color_dict),
        )

    n.loads_t.p_set.iloc[0:timesteps].filter(regex='C&I Load').plot(ax=ax, c="k")

    ax.set_ylim([-1e4,1e4])


plot_dispatch(cfe, ci_label='PPA', timesteps=24*4)

In [10]:
components = ['generators', 'links', 'storage_units']
for c in components:
    print( getattr(network, c)['p_nom'] )

Generator
MYSPE-bioenergy-unspecified-ext-2030            0.00
MYSPE-combined-cycle-ext-2030               12669.20
MYSPE-coal-subcritical-ext-2030              8200.00
MYSPE-coal-ultrasupercritical-ext-2030       4000.00
MYSPE-hydro-unspecified-ext-2030             2536.10
MYSPE-open-cycle-gas-turbine-ext-2030         834.00
MYSPE-photovoltaic-unspecified-ext-2030      1445.42
MYSPE-waste-ext-2030                            0.00
MYSPE-wind-offshore-unspecified-ext-2030        0.00
MYSPE-wind-onshore-ext-2030                     0.00
Name: p_nom, dtype: float64
Series([], Name: p_nom, dtype: float64)
StorageUnit
MYSPE-lithium-ion-ext-2030    0.0
Name: p_nom, dtype: float64


In [18]:
n = pypsa.Network('/Users/aman/Library/CloudStorage/GoogleDrive-aman.m@transitionzero.org/My Drive/tz-pypsa-workspace/tza-google-cfe/run/N_BROWNFIELD.nc')

INFO:pypsa.io:Imported network N_BROWNFIELD.nc has buses, carriers, generators, links, loads, storage_units


In [27]:
n.links

,bus0,bus1,carrier,p_nom_extendable,capital_cost,marginal_cost,p_nom_opt,type,efficiency,build_year,...,start_up_cost,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down
Link,,,,,,,,,,,,,,,,,,,,,
MYSPE C&I Grid Imports,MYSPE,MYSPE C&I Grid,AC,True,0.01,0.01,2640.36638,,1.0,0,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
MYSPE C&I Grid Exports,MYSPE C&I Grid,MYSPE,AC,False,0.01,0.01,0.00000,,1.0,0,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
MYSPE C&I Storage Charge,MYSPE C&I Grid,MYSPE C&I Storage,AC,False,0.01,0.01,0.00000,,1.0,0,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
MYSPE C&I Storage Discharge,MYSPE C&I Grid,MYSPE C&I Storage,AC,False,0.01,0.01,0.00000,,1.0,0,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0


In [69]:
import pypsa
import logging
import pandas as pd

logging.basicConfig(level=logging.CRITICAL + 1)
logging.getLogger("gurobipy").disabled = True
logging.getLogger("linopy").disabled = True
logging.getLogger("pypsa").disabled = True

def MakeNetwork():

    n = pypsa.Network()

    # snapshots
    n.set_snapshots(range(4))

    # carriers
    n.add("Carrier", "Dirty", co2_emissions=0.2)
    n.add("Carrier", "Clean", co2_emissions=0.0)

    # add bus
    n.add("Bus", "AnyTown Grid", v_nom=20.)
    n.add("Bus", "C&I Grid", v_nom=20.)
    n.add("Bus", "C&I Storage", v_nom=20.)

    # add demands
    n.add("Load", "AnyTown Load", bus="AnyTown Grid", p_set=[6,6,6,6])
    n.add("Load", "C&I Load", bus="C&I Grid", p_set=[2,5,3,1])

    # add generators
    n.add(
        "Generator", 
        "AnyTown Dirty Gen", 
        bus="AnyTown Grid", 
        carrier="Dirty",
        p_nom=10, 
        marginal_cost=10, 
        capital_cost=1e6, 
        p_nom_extendable=True
    )

    n.add(
        "Generator", 
        "AnyTown Clean Gen", 
        bus="AnyTown Grid", 
        carrier="Clean",
        p_nom=10, 
        marginal_cost=15, 
        capital_cost=1e6, 
        p_max_pu=[1,1,0,0], 
        p_nom_extendable=True
    )

    n.add(
        "Generator", 
        "C&I PPA", 
        bus="C&I Grid",
        carrier="Clean",
        p_nom=0, 
        marginal_cost=15, 
        capital_cost=1e6, 
        p_max_pu=[0.8,0.,0.3,0.7], 
        p_nom_extendable=True,
    )

    # add storage
    n.add(
        "StorageUnit",
        "C&I PPA Storage",
        bus="C&I Storage",
        p_nom=0,
        p_nom_extendable=True,
        cyclic_state_of_charge=True,
        capital_cost=1e9,
        marginal_cost=0,
        max_hours=4,
        efficiency_store=0.9,
        efficiency_dispatch=0.9,
    )

    # add links
    n.add(
        "Link",
        "C&I ImportFromAnyTown",
        bus0="AnyTown Grid", 
        bus1="C&I Grid", 
        p_nom=1e6,
        marginal_cost=1,
    )

    n.add(
        "Link",
        "C&I ExportToAnyTown",
        bus0="C&I Grid", 
        bus1="AnyTown Grid", 
        p_nom=1e6,
        marginal_cost=1,
    )

    n.add(
        "Link",
        "PPA_StorageCharge",
        bus0="C&I Grid", 
        bus1="C&I Storage", 
        p_nom=0,
        p_nom_extendable=True,
        marginal_cost=1,
    )

    n.add(
        "Link",
        "PPA_StorageDischarge",
        bus0="C&I Storage", 
        bus1="C&I Grid", 
        p_nom=0,
        p_nom_extendable=True,
        marginal_cost=1,
    )

    return n

brownfield = MakeNetwork()
# optimise
brownfield.optimize(solver_name='gurobi', solver_options={'log_to_console': False})

res_100 = MakeNetwork()

res_100.optimize.create_model()

# add 100% RES constraint
sum_ci_load = res_100.loads_t.p_set['C&I Load'].sum()

sum_ppa_procured = (
    res_100
    .model
    .variables['Generator-p']
    .sel(
        Generator='C&I PPA'
        )
    .sum()
)

res_100.model.add_constraints(
    sum_ppa_procured >= sum_ci_load,
    name = '100_RES_constraint',
)

res_100.optimize.solve_model(solver_name='gurobi', solver_options={'log_to_console': False})

cfe = MakeNetwork()

cfe.optimize.create_model()

CFE_TARGET = 0.9
MAXIMUM_EXCESS = 0.2

# Constraint 1: Hourly matching
#   CI_Demand[t] + PPA_StorageCharge[t] - PPA_StorageDischarge[t] = PPA[t] - Excess[t] + GridSupply[t]

CI_Demand = cfe.loads_t.p_set['C&I Load'].values
CI_StorageCharge = cfe.model.variables['Link-p'].sel(Link='PPA_StorageCharge')
CI_StorageDischarge = cfe.model.variables['Link-p'].sel(Link='PPA_StorageDischarge')
CI_PPA = cfe.model.variables['Generator-p'].sel(Generator='C&I PPA')
CI_Export = cfe.model.variables['Link-p'].sel(Link='C&I ExportToAnyTown')
CI_GridImport = cfe.model.variables['Link-p'].sel(Link='C&I ImportFromAnyTown')

# cfe.model.add_constraints(
#     ((CI_StorageCharge - CI_StorageDischarge) + CI_Demand) == CI_PPA - CI_Export + CI_GridImport,
#     name = 'Hourly_matching_constraint',
# )

# Constraint 2: CFE target
#   SUM( PPA[t] - Excess[t] + GridSupply[t]*GridCFE[t] ) / SUM( CI_Demand[t] ) >= CFE_target

GRID_CFE = (brownfield.generators_t.p['AnyTown Clean Gen'] / brownfield.generators_t.p['AnyTown Dirty Gen']).values

cfe.model.add_constraints(
    (CI_PPA - CI_Export + CI_GridImport * 0.3).sum() == 10,
    #(CI_PPA - CI_Export + CI_GridImport * list(GRID_CFE)).sum() == ((CI_StorageCharge - CI_StorageDischarge) + CI_Demand).sum() * CFE_TARGET,
    name = 'CFE_target_constraint',
)


# # Constraint 3: Total excess
# cfe.model.add_constraints(
#     CI_Export.sum() <= sum(CI_Demand) * MAXIMUM_EXCESS,
#     name = 'total_excess_constraint',
# )

cfe.optimize.solve_model(solver_name='gurobi', solver_options={'log_to_console': False})

INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.model:Solver options:
 - log_to_console: False


Set parameter Username


INFO:gurobipy.gurobipy:Set parameter Username


Academic license - for non-commercial use only - expires 2025-09-24


INFO:gurobipy.gurobipy:Academic license - for non-commercial use only - expires 2025-09-24
INFO:linopy.io: Writing time: 0.03s


...

Reading time = 0.00 seconds


INFO:gurobipy.gurobipy:Reading time = 0.00 seconds


obj: 103 rows, 47 columns, 195 nonzeros


INFO:gurobipy.gurobipy:obj: 103 rows, 47 columns, 195 nonzeros
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 47 primals, 103 duals
Objective: 1.97e+09
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-fix-p-lower, Link-fix-p-upper, Link-ext-p-lower, Link-ext-p-upper, StorageUnit-ext-p_dispatch-lower, StorageUnit-ext-p_dispatch-upper, StorageUnit-ext-p_store-lower, StorageUnit-ext-p_store-upper, StorageUnit-ext-state_of_charge-lower, StorageUnit-ext-state_of_charge-upper, StorageUnit-energy_balance, CFE_target_constraint were not assigned to the network.


('ok', 'optimal')

In [68]:
CI_GridImport

snapshot
0    0.000000
1    3.031496
2    0.000000
3    0.000000
dtype: float64

In [41]:
cfe.buses

attribute,v_nom,type,x,y,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,generator,sub_network
Bus,,,,,,,,,,,,
AnyTown Grid,20.0,,0.0,0.0,AC,,1.0,0.0,inf,PQ,,
C&I Grid,20.0,,0.0,0.0,AC,,1.0,0.0,inf,PQ,,
C&I Storage,20.0,,0.0,0.0,AC,,1.0,0.0,inf,PQ,,


In [62]:
# (CI_PPA - CI_GridExport + (CI_GridImport * GridCFE)).sum() \
#     >= ((CI_StorageCharge - CI_StorageDischarge) + CI_Demand).sum() * CFE_Score

N_CFE = cfe.copy()
GridCFE = GRID_CFE 
CFE_Score = CFE_TARGET
ci_identifier = 'C&I'

CI_PPA = N_CFE.generators_t.p[[i for i in N_CFE.generators.index if ci_identifier in i and 'PPA' in i]].sum(axis=1)
CI_GridExport = N_CFE.links_t.p0[[i for i in N_CFE.links.index if ci_identifier in i and 'Export' in i]].sum(axis=1)
CI_GridImport = N_CFE.links_t.p0[[i for i in N_CFE.links.index if ci_identifier in i and 'Import' in i]].sum(axis=1)
LHS = (CI_PPA - CI_GridExport + CI_GridImport * GridCFE).sum()

CI_Demand = N_CFE.loads_t.p_set.filter(regex=ci_identifier).values.flatten()
CI_StorageCharge = N_CFE.links_t.p0[[i for i in N_CFE.links.index if ci_identifier in i and 'Charge' in i]].sum(axis=1)
CI_StorageDischarge = N_CFE.links_t.p0[[i for i in N_CFE.links.index if ci_identifier in i and 'Discharge' in i]].sum(axis=1)
RHS = ((CI_StorageCharge - CI_StorageDischarge) + CI_Demand).sum() * CFE_Score

if LHS >= RHS:
    print('Constraint met')
else:
    print('Constraint breached')

Constraint breached


In [63]:
LHS

9.090551181103493